# Reservoir Computing for Covid dataset

## Create a reservoir

In [ ]:
# Create, train and evaluate ANN_Dense models
# on all fluxes simulated training set
# Repeat the process with different seeds
# This cell takes several hours to execute
# Save the best model in a reservoir

from Library.Import import *
from Library.Build_Dataset import TrainingSet, get_index_from_id
from Library.Build_Model import Neural_Model, model_input
from Library.Build_Model import evaluate_model, train_evaluate_model
from sklearn.metrics import r2_score
import cobra

DIRECTORY = './Dataset_input/Covid/'
seed = 10
np.random.seed(seed=seed)
model_type = 'ANN_Dense' # 'AMN_QP' 'ANN_Dense'
trainname = 'Covid'
cobrafile = f'{DIRECTORY}{trainname}_duplicated.xml'
cobramodel = cobra.io.read_sbml_model(cobrafile)
obj = get_index_from_id('BIOMASS_Ec_iML1515_core_75p37M', cobramodel.reactions) 
KOs = ['WT', 'hisD', 'proC', 'serC', 'tyrA', 'trpA', 'pheA']
reduce = 0

for ko in ['WT']: # KOs:
    
    trainingfile = f'{DIRECTORY}{trainname}_{ko}_train'
    reducefile = f'{DIRECTORY}{trainname}_{ko}_reduce'
    reservoirname = f'{trainname}_{ko}_{model_type}'
    reservoirfile = f'{DIRECTORY}{reservoirname}_{str(seed)}' 
    
    if reduce:  # remove fluxes < reduce value
        parameter = TrainingSet()
        parameter.load(trainingfile)
        parameter.save(reducefile, reduce=reduce, verbose=True)
        parameter.load(reducefile)
        parameter.printout()
        trainingfile = reducefile
        

    Maxloop, Q2, PRED = 1, [], []
    for Nloop in range(Maxloop):
        
        model = Neural_Model(trainingfile=trainingfile,
                         model_type=model_type,
                         scaler=True,
                         n_hidden=2, hidden_dim=500, # 1000
                         epochs = 5000, # 500
                         train_rate=1e-3,
                         batch_size=100, # 200, 100, 10
                         xfold=5, 
                         niter=1,
                         verbose=False)
        
        #idx = np.random.randint(model.X.shape[0], size=1000)
        #model.X, model.Y = model.X[idx,:], model.Y[idx,:]
        
        # Train and evaluate
        start_time = time.time()
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
        delta_time = time.time() - start_time

        # Printing cross-validation results
        stats.printout(reservoirname, delta_time)

        r2 = r2_score(model.Y[:,obj], pred[:,obj], multioutput='variance_weighted')
        print(f'Iter {Nloop} Collated Q2 growth {r2:.4f}')
        r2 = r2_score(model.Y, pred[:,:model.Y.shape[1]], multioutput='variance_weighted')
        print(f'Iter {Nloop} Collated Q2 all {r2:.4f}')
        Q2.append(r2)
        PRED.append(pred[:, 0])
        if r2 == max(Q2):  # save the best model
            reservoir.save(reservoirfile)

    # Some printing
    Q2, PRED = np.asarray(Q2), np.asarray(PRED)
    print(f'{trainname} Averaged Q2 = {np.mean(Q2):4f} ± {np.std(Q2):.4f} Best Q2 = {np.max(Q2):.4f}')

    # Verification
    reservoir = Neural_Model()
    reservoir.load(reservoirfile, output_dim=1)
    reservoir.printout()
    X, Y = model_input(reservoir, verbose=False)
    pred, _ = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    y = pred[:,:Y.shape[1]]     
    r2 = r2_score(Y[:,0], y[:,0], multioutput='variance_weighted')
    print(f'Final R2 growth {r2:.4f}')


ANN Dense scaler 4.0
-------train (8000, 56) (8000, 3682)
-------test  (2000, 56) (2000, 3682)
----------------------------------- ANN_Dense
Dense layer n_hidden: 2 hidden_dim: 500 input_dim: 56 output_dim: 3682 activation: relu trainable: True
nbr parameters: 2123682
250/250 [==============================] - 0s 964us/step


In [ ]:
# Create, train and evaluate AMN_QP models
# on GR FBA simulated training set
# Repeat the process with different seeds
# This cell takes several hours to execute
# Save the best model in a reservoir

from Library.Import import *
from Library.Build_Dataset import TrainingSet, get_index_from_id
from Library.Build_Model import Neural_Model, model_input
from Library.Build_Model import evaluate_model, train_evaluate_model
from sklearn.metrics import r2_score
import cobra

DIRECTORY = './Dataset_input/Covid/'
seed = 10
np.random.seed(seed=seed)
model_type = 'AMN_QP' # 'AMN_QP' 'ANN_Dense'
trainname = 'Covid'
reduce = 0
KOs = ['WT', 'hisD', 'proC', 'serC', 'tyrA', 'trpA', 'pheA']

for ko in ['WT']: # KOs:

    trainingfile = f'{DIRECTORY}{trainname}_{ko}_train'
    reducefile = f'{DIRECTORY}{trainname}_{ko}_reduce'
    reservoirname = f'{trainname}_{ko}_{model_type}'
    reservoirfile = f'{DIRECTORY}{reservoirname}_{str(seed)}' 
    
    if reduce:  # remove fluxes < reduce value
        parameter = TrainingSet()
        parameter.load(trainingfile)
        parameter.save(reducefile, reduce=reduce, verbose=True)
        parameter.load(reducefile)
        parameter.printout()
        trainingfile = reducefile
    
    Maxloop, Q2, PRED = 1, [], []
    for Nloop in range(Maxloop):
        
        model = Neural_Model(trainingfile=trainingfile,
                         objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
                         model_type=model_type,
                         scaler=True,
                         n_hidden=1, hidden_dim=500, # 1000
                         output_dim=1,
                         epochs=1000, # 500
                         batch_size=100, # 200, 100, 10
                         xfold=5, 
                         verbose=False)

        # Train and evaluate
        start_time = time.time()
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
        delta_time = time.time() - start_time

        # Printing cross-validation results
        stats.printout(reservoirname, delta_time)

        r2 = r2_score(model.Y[:,0], pred[:,0], multioutput='variance_weighted')
        print(f'Iter {Nloop} Collated Q2 growth {r2:.4f}')
        r2 = r2_score(model.Y, pred[:,:model.Y.shape[1]], multioutput='variance_weighted')
        print(f'Iter {Nloop} Collated Q2 all {r2:.4f}')
        Q2.append(r2)
        PRED.append(pred[:, 0])
        if r2 == max(Q2):  # save the best model
            reservoir.save(reservoirfile)

    # Some printing
    Q2, PRED = np.asarray(Q2), np.asarray(PRED)
    print(f'{trainname} Averaged Q2 = {np.mean(Q2):4f} ± {np.std(Q2):.4f} Best Q2 = {np.max(Q2):.4f}')

    # Verification
    reservoir = Neural_Model()
    reservoir.load(reservoirfile, output_dim=1)
    reservoir.printout()
    X, Y = model_input(reservoir, verbose=False)
    pred, _ = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    y = pred[:,:Y.shape[1]]     
    r2 = r2_score(Y[:,0], y[:,0], multioutput='variance_weighted')
    print(f'Final R2 growth {r2:.4f}')


AMN scaler: 4.0
QP input shape: (10000, 56) (10000, 4)
-------train (8000, 56) (8000, 4)
-------test  (2000, 56) (2000, 4)
----------------------------------- AMN_QP
Dense layer n_hidden: 1 hidden_dim: 500 input_dim: 56 output_dim: 3682 activation: relu trainable: True
Instructions for updating:
Use `tf.linalg.matmul` instead
AMN output shapes for PoutV: (None, 1) SV: (None, 1) Pin: (None, 1) Pko: (None, 1)  V: (None, 3682) outputs: (None, 3686)
nbr parameters: 1873182
250/250 [==============================] - 1s 3ms/step
Loss Vout: 2.6E-02
Loss SV:   3.1E-02
Loss Vin:  3.4E-04
Loss Vko:  0.0E+00


## Testing reservoir with provided patient MS data

In [ ]:
# Running a AMN reservoir in predictve mode with a test

from Library.Import import *
from Library.Utilities import bayes_classifier, LeaveXout, read_XY
from Library.Utilities import best_accuracy_threshold
from Library.Build_Model import Neural_Model, model_input
from Library.Build_Model import evaluate_model, train_evaluate_model
from sklearn.metrics import r2_score, accuracy_score
    
DIRECTORY = './Dataset_input/Covid/'
seed = 10
np.random.seed(seed=seed)
model_type = 'AMN_QP' # 'AMN_QP' 'ANN_Dense'
trainname = 'Covid'
testfile = f'{DIRECTORY}Covid_MS'
feature, X_test, y_test_true = read_XY(testfile, nY=1, scaling='X')
xfold = 10
niter = 3
learner = bayes_classifier # To classify based on all phenotypes
selection = True # To reduce the number of features when performing Leave X out

KOs = ['WT', 'hisD', 'proC', 'serC', 'tyrA', 'trpA', 'pheA']

for ko in ['trpA']: # in KOs:
    
    trainingfile = f'{DIRECTORY}{trainname}_{ko}_train'
    reservoirname = f'{trainname}_{ko}_{model_type}'
    reservoirfile = f'{DIRECTORY}{reservoirname}_{str(seed)}'
    
    # Verify the reservoir
    reservoir = Neural_Model()
    reservoir.load(reservoirfile, output_dim=1)
    reservoir.printout()
    feature = [r.id for r in reservoir.cobramodel.reactions]
    X, Y = model_input(reservoir, verbose=False)
    pred, _ = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    y = pred[:,:Y.shape[1]]     
    r2 = r2_score(Y[:,0], y[:,0], multioutput='variance_weighted')

    # Predict for new values
    pred = reservoir.model.predict(X_test)
    L = reservoir.number_constraint+1
    X = pred[:,L:]
    X[X < 1e-3] = 0
    zero_columns = np.argwhere(np.all(X == 0, axis=0)).flatten()
    X = np.delete(X, zero_columns, axis=1)
    feature = np.delete(feature, zero_columns, axis=0)
    
    # Accuracy based on growth rate
    y_pred = pred[:,:1]    
    acc, treshold = best_accuracy_threshold(y_pred.ravel(), y_test_true.ravel())
    print(f'KO: {ko} Growth Rate Threshold: {treshold:.3f} Acc: {acc:.2f}')

    # Accuracy based on phenotype
    acc_avr, acc_dev, feature = LeaveXout(X, y_test_true.ravel(), feature, 
                                     learner=learner, regression=False, 
                                     xfold=xfold, niter=niter, 
                                     selection=selection, verbose=True)

    # Printing
    F = np.array2string(feature).replace('[','').replace(']','')
    print(f'KO {ko} Phenotype Size: {X.shape} Method: {learner.__name__} '
          f'Acc: {acc_avr:.3f}±{acc_dev:.3f} '
          f'for {xfold}-fold-CV and {niter} iter\n'
          f'Selected features {len(feature)}: {F}')


training file: ./Dataset_input/Covid/Covid_trpA_train
model type: AMN_QP
model number constraints: 3
model scaler: 4.0
model input dim: 56
model output dim: 1
training set size (500, 56) (500, 1)
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
training epochs: 500
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 10
training validation iter: 0
training xfold: 5
3/3 [==============================] - 0s 4ms/step
(81, 3682) 3682
KO: trpA Growth Rate Threshold: 0.181 Acc: 0.62
Size: 458 Remove: PYNP2r_for Score: 0.506
Size: 457 Remove: DURIPP_for Score: 0.506
Size: 456 Remove: MAN6PI_for Score: 0.506
Size: 455 Remove: PPM_for Score: 0.506
Size: 454 Remove: EX_pi_e_o Score: 0.506
Size: 453 Remove: PYK Score: 0.506
Size: 452 Remove: EX_co2_e_o Score: 0.506
Size: 451 Remove: A5PISO_for Score: 0.506
Size: 450 Remove: GTHOr_for Score: 0.506
Size: 449 Remove: ORPT_for Score: 0.506
Size: 448 Remove: DHAD2 Score: 0.506
Size: 447 

In [2]:
# Running a Cobra reservoir in predictve mode with a test set 

import cobra
from Library.Import import *
from Library.Utilities import bayes_classifier, LeaveXout, read_XY
from Library.Utilities import best_accuracy_threshold
from Library.Build_Dataset import TrainingSet, create_medium_run_cobra
from sklearn.metrics import r2_score, accuracy_score
    
DIRECTORY = './Dataset_input/Covid/'
seed = 10
np.random.seed(seed=seed)
trainname = 'Covid'
testfile = f'{DIRECTORY}Covid_MS'
feature, X_test, y_test_true = read_XY(testfile, nY=1, scaling='X')
xfold = 10
niter = 3
learner = bayes_classifier # To classify based on all phenotypes
selection = False # To reduce the number of features when performing Leave X out

KOs = ['WT', 'hisD', 'proC', 'serC', 'tyrA', 'trpA', 'pheA']

for ko in KOs:
    
    trainingfile = f'{DIRECTORY}{trainname}_{ko}_train'
    parameter = TrainingSet()
    parameter.load(trainingfile)
    print(trainingfile)
    parameter.printout()
    
    # Run cobra to get growth rate for X_test
    y_pred, X = create_medium_run_cobra(parameter.model, 
                                           parameter.objective, 
                                           parameter.medium, 
                                           X_test, method='FBA', scaler=1, 
                                           genekos=[ko], verbose=False) 
    feature = [r.id for r in parameter.model.reactions]
    X[X < 1e-3] = 0
    zero_columns = np.argwhere(np.all(X == 0, axis=0)).flatten()
    X = np.delete(X, zero_columns, axis=1)
    feature = np.delete(feature, zero_columns, axis=0)
    
    # Accuracy based on growth rate
    acc, treshold = best_accuracy_threshold(y_pred, y_test_true.ravel())
    print(f'KO: {ko} Growth Rate Threshold: {treshold:.3f} Acc: {acc:.3f}')

    # Accuracy based on phenotype

    acc_avr, acc_dev, feature = LeaveXout(X, y_test_true.ravel(), feature, 
                                     learner=learner, regression=False, 
                                     xfold=xfold, niter=niter, 
                                     selection=selection, verbose=False)
    
    # Printing
    F = np.array2string(feature).replace('[','').replace(']','')
    print(f'KO {ko} Phenotype Size: {X.shape} Method: {learner.__name__} '
          f'Acc: {acc_avr:.3f}±{acc_dev:.3f} '
          f'for {xfold}-fold-CV and {niter} iter\n'
          f'Selected features {len(feature)}: {F}')


./Dataset_input/Covid/Covid_WT_train
model file name: ./Dataset_input/Covid/Covid_WT_train
reduced model: False
medium file name: ./Dataset_input/Covid/Covid
medium: 80
experimental medium: 0
list of reactions in objective: ['BIOMASS_Ec_iML1515_core_75p37M']
method: FBA
trainingsize: 500
list of measured reactions: 3682
Stoichiometric matrix: (1877, 3682)
Boundary matrix from reactions to medium: (56, 3682)
KO matrix from reactions to ko: (0, 3682)
Measurement matrix from reaction to measures: (3682, 3682)
Training set X: (500, 56)
Training set Y: (500, 3682)
KO: WT Growth Rate Threshold: 0.046 Acc: 0.617
KO WT Phenotype Size: (81, 515) Method: bayes_classifier Acc: 0.691±0.017 for 10-fold-CV and 3 iter
Selected features 515: 'SHK3Dr_for' 'PYNP2r_for' 'G5SD' 'ICDHyr_for' 'PPA' 'TRPAS2_for'
 'ALAR_for' 'ALATA_L_for' 'PPM_for' 'ASPTA_for' 'EX_pi_e_o' 'PYK'
 'EX_co2_e_o' 'A5PISO_for' 'VALTA_for' 'ACHBS' 'PFK_3' 'DHAD2' 'ACLS'
 'PSCVT_for' 'PFL' 'FRD2' 'CHORM' 'PTAr_for' 'CHORS' 'EX_hxan_e

## Predict ODmax with various learner

In [1]:
# Predict ODmax with various learner

from Library.Import import *
from Library.Utilities import Linear, MLP, XGB, LeaveXout, read_XY
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

DIRECTORY = './'
problempath = f'{DIRECTORY}Dataset_input/Covid/'
problem = 'Covid_ODMAX'
xfold = 14
niter = 5
learner = Linear
regression = True
filename = f'{problempath}{problem}'
feature, X, y = read_XY(filename, scaling='')

# Train X, y for regression
r2_avr, r2_dev, feature = LeaveXout(X, y.ravel(), feature, 
                                     learner=learner, regression=regression, 
                                     xfold=xfold, niter=niter, 
                                     selection=True, verbose=True)

F = np.array2string(feature[:-1]).replace('[','').replace(']','')
print(f'{problem} Size: {X.shape} Method: {learner.__name__} '
      f'R2: {r2_avr:.3f}±{r2_dev:.3f} '
      f'for {xfold}-fold-CV and {niter} iter\n'
      f'Selected features {len(feature)}: {F}')


Size: 68 Remove: EX_galctn__L_e_i Score: 0.046
Size: 67 Remove: EX_acnam_e_i Score: 0.146
Size: 66 Remove: EX_gln__L_e_i Score: 0.143
Size: 65 Remove: EX_urate_e_i Score: 0.174
Size: 64 Remove: EX_quin_e_i Score: 0.201
Size: 63 Remove: EX_csn_e_i Score: 0.274
Size: 62 Remove: EX_succ_e_i Score: 0.355
Size: 61 Remove: EX_cys__D_e_i Score: 0.354
Size: 60 Remove: EX_his__L_e_i Score: 0.380
Size: 59 Remove: EX_taur_e_i Score: 0.419
Size: 58 Remove: EX_asp__L_e_i Score: 0.408
Size: 57 Remove: EX_ddca_e_i Score: 0.507
Size: 56 Remove: EX_4abut_e_i Score: 0.571
Size: 55 Remove: EX_pppn_e_i Score: 0.631
Size: 54 Remove: EX_5mtr_e_i Score: 0.601
Size: 53 Remove: EX_xylu__L_e_i Score: 0.651
Size: 52 Remove: EX_ser__D_e_i Score: 0.603
Size: 51 Remove: EX_arab__L_e_i Score: 0.672
Size: 50 Remove: EX_gam6p_e_i Score: 0.691
Size: 49 Remove: EX_hom__L_e_i Score: 0.808
Size: 48 Remove: EX_mal__D_e_i Score: 0.764
Size: 47 Remove: EX_pro__L_e_i Score: 0.725
Size: 46 Remove: EX_inost_e_i Score: 0.788
Siz

## Run RC to predict growth rate from metabolite MS signals

In [1]:
# Create, train and evaluate RC models
# on experimental data to create a file for Cobra

from Library.Import import *
from Library.Build_Reservoir import RC_run, RC_write_multiple

DIRECTORY = './Dataset_input/Covid/'

seed = 10
np.random.seed(seed=seed)
xfold = 5
repeat = 1 # 3
train_rate = 1.0e-4
epochs = 1000
n_hidden_prior = 1 # -1 binary input,  >0 ANN 
hidden_dim_prior = int(39*20)
activation_prior='relu' # 'sharp_sigmoid' 
failure = 10
multiple = 1 # -1 no stats > 0 nbr of reservoirs to get stats 

trainingfile  = f'{DIRECTORY}COVIDXY'
reservoirfile = f'{DIRECTORY}COVIDEXP_1200_AMN_QP_10' 
resultfile    = f'{DIRECTORY}COVIDXY_for_Cobra'

H, X, Y = read_XY(trainingfile, nY=1)
start_time = time.time()
model, pred, R2_avr, R2_dev, Q2_avr, Q2_dev, Med = \
RC_run(reservoirfile, X, Y,
               mode='AMN_objective',
               n_hidden_prior=n_hidden_prior, 
               hidden_dim_prior=hidden_dim_prior,
               activation_prior=activation_prior,
               train_rate=train_rate, 
               failure=failure,
               repeat=repeat, xfold=xfold, epochs=epochs, verbose=True)
delta_time = time.time() - start_time
print(f'R2: {R2_avr:.2f} ± {R2_dev:.4f} Q2: {Q2_avr:.2f} ± {Q2_dev:.4f} cpu time {delta_time:.2f}')

RC_write_multiple(reservoirfile, resultfile, model, Y, pred, multiple=multiple, verbose=False)


Instructions for updating:
Use `tf.linalg.matmul` instead
number of reactions: 3682=3682
number of metabolites: 1877
filtered measurements size: 1
RC reservoir file: ./Dataset_input/Covid/COVIDEXP_1200_AMN_QP_10
RC model type: RC
RC number constraint: 1
RC precsion: 0
RC model input dim: 38
RC model output dim: 1
training set size (120, 38) (120, 1)
reservoir S, Pin, Pout matrices (1877, 3682) (38, 3682) (1, 3682)
RC training epochs: 1000
RC training regression: True
RC training learn rate: 0.0001
RC training dropout: 0.25
RC training batch size: 10
RC training validation iter: 0
RC training xfold: 5
--------prior network --------
training file: None
model type: ANN_Dense
model number constraints: 0
model scaler: 0.0
model input dim: 38
model output dim: 38
no training set provided
nbr hidden layer: 1
hidden layer size: 780
activation function: relu
--------reservoir network-----
training file: ./Dataset_input/Covid/COVIDEXP_1200
model type: AMN_QP
model number constraints: 3
model sca